<a href="https://colab.research.google.com/github/DiptoSarker/Bangla_Fruits_Classification/blob/main/BanglaFruitsClassifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import os
import numpy as np

In [3]:
base_dir=r"/content/drive/MyDrive/fruit"

In [4]:
IMAGE_SIZE=224
BATCH_SIZE=64

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(


    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1)

validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,

    validation_split=0.1
)

In [5]:
train_genarator=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'


)

validation_generator=validation_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'


)


Found 2907 images belonging to 29 classes.
Found 312 images belonging to 29 classes.


In [6]:
!pip install tensorflow

In [7]:
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from glob import glob

In [8]:
IMAGE_SIZE=[224,224]
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
vgg.output

58889256/58889256 [==============================] - 0s 0us/step


<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [9]:
for layer in vgg.layers:
    layer.trainable=False

In [10]:
folders=glob(r"/content/drive/MyDrive/fruit/*")
print(len(folders))

29


In [11]:
x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
epoch=15

history=model.fit(train_genarator,
                  steps_per_epoch=len(train_genarator),
                  epochs=epoch,
                  validation_data=validation_generator,
                  validation_steps=len(validation_generator)
                 )

Epoch 1/15
 5/46 [==>...........................] - ETA: 29:46 - loss: 4.2838 - accuracy: 0.0625

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


46/46 [==============================] - 2318s 50s/step - loss: 2.4775 - accuracy: 0.3794 - val_loss: 1.5747 - val_accuracy: 0.5513
Epoch 2/15
46/46 [==============================] - 2271s 49s/step - loss: 1.0116 - accuracy: 0.7110 - val_loss: 1.1737 - val_accuracy: 0.6859
Epoch 3/15
46/46 [==============================] - 2246s 49s/step - loss: 0.6503 - accuracy: 0.8211 - val_loss: 1.1972 - val_accuracy: 0.6891
Epoch 4/15
46/46 [==============================] - 2202s 48s/step - loss: 0.5263 - accuracy: 0.8490 - val_loss: 1.2117 - val_accuracy: 0.6859
Epoch 5/15
46/46 [==============================] - 2265s 49s/step - loss: 0.3505 - accuracy: 0.9099 - val_loss: 0.9605 - val_accuracy: 0.7340
Epoch 6/15
46/46 [==============================] - 2256s 49s/step - loss: 0.2529 - accuracy: 0.9408 - val_loss: 1.0236 - val_accuracy: 0.7596
Epoch 7/15
46/46 [==============================] - 2209s 48s/step - loss: 0.2186 - accuracy: 0.9501 - val_loss: 0.9374 - val_accuracy: 0.7500
Epoch 8/15

In [14]:
model.save('/content/my_model.h5')

import shutil

# Move the model file to your Google Drive
shutil.move('/content/my_model.h5', '/content/drive/MyDrive/fruit')


'/content/drive/MyDrive/fruit/my_model.h5'

In [27]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_pred = image.load_img('/content/drive/MyDrive/fruit/boroi/download (9).jpg', target_size=(224, 224))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred, axis=0)
result = model.predict(img_pred)
print(result)

1/1 [==============================] - 1s 557ms/step
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]


In [28]:
predicted_class = np.argmax(result, axis=1)  # Get the index of the class with highest probability

# Assuming you have a list of class names
class_names = ['amra', 'dalim', 'akh','am','anarosh', 'angur', 'apple', 'bael', 'boroi','cherry','chickoo fruit',
               'dragon fol', 'jam', 'jambura','kathal','kamranga','khejur','kiwi fol','komola','kodbel','kola',
               'lichu','lotkon','narkel', 'nashpati','peyara','strawberry','tetul','tormuj' ]  # Add your class names here

prediction = class_names[predicted_class[0]]
print(prediction)

boroi
